In [9]:
import scipy.io

#Numpyfile = scipy.io.loadmat('AllSamples.mat')

FileData = load('AllSamples.mat');
csvwrite('AllSamples.csv', FileData.M);

NameError: name 'load' is not defined

In [ ]:
# Numpyfile

Actual K-Means Clustering Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#, mpld3
from matplotlib import style
import pandas as pd 

style.use('ggplot')

import pandas as pd

In [ ]:
class K_Means:
    def __init__(self, paramTup, k =3, tolerance = 0.0001, max_iterations = 500):
        self.k = k
        self.tolerance = tolerance
        self.max_iterations = max_iterations
        self.centroids = {}
        for i in range(self.k):
            self.centroids[i] = paramTup[i]


    def fit(self, data):
        #(!!!) CHOSE FIRST K ELEMENTS AS CENTROIDS. MANUALLY INITIALIZE!!
        #initialize the centroids, the first 'k' elements in the dataset will be our initial centroids
        

        #begin iterations
        for i in range(self.max_iterations):
            self.classes = {}
            for i in range(self.k):
                self.classes[i] = []

            #find the distance between the point and cluster; choose the nearest centroid
            for features in data:
                distances = [np.linalg.norm(features - self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classes[classification].append(features)

            previous = dict(self.centroids)

            #average the cluster datapoints to re-calculate the centroids
            for classification in self.classes:
                self.centroids[classification] = np.average(self.classes[classification], axis = 0)

            isOptimal = True

            for centroid in self.centroids:

                original_centroid = previous[centroid]
                curr = self.centroids[centroid]

                if np.sum((curr - original_centroid)/original_centroid * 100.0) > self.tolerance:
                    isOptimal = False

            #break out of the main loop if the results are optimal, ie. the centroids don't change their positions much(more than our tolerance)
            if isOptimal:
                break

    def pred(self, data):
        distances = [np.linalg.norm(data - self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

In [ ]:
def main():
    filepath = 'AllSamples.csv'
    df = pd.read_csv(filepath, sep=',')

    #Check output of CSV
    print(df.shape)
    df

    df = df[['X', 'Y']]
    dataset = df.astype(float).values.tolist()

    #Return Numpy Array From CSV
    X = df.values
    X
    
    
    tripleCent = [[(2,8), (3,3), (5,8)], [(2,8),(3,3),(6,4)], [(2,8),(1,2),(6,4)]]
    for i in range(3):
        km = K_Means(tripleCent[i], 3)
        km.fit(X)
        print("FINAL CENTROIDS ARE: " + str(km.centroids))

        # Plotting starts here, the colors
        colors = 10*["r", "g", "c", "b", "k"]

        for centroid in km.centroids:
            plt.scatter(km.centroids[centroid][0], km.centroids[centroid][1], s = 130, marker = "x")

        for classification in km.classes:
            color = colors[classification]
            for features in km.classes[classification]:
                plt.scatter(features[0], features[1], color = color,s = 30)

        print("KMC graph for " + str(tripleCent[i]))
        plt.show()
    
if __name__ == "__main__":
    main()